# Hello F#

This is a .net interactive notebook with some sample code of the basic of F# that you play around with.

Things we will cover in this notebook is:

* Simple types
* What is `let`
* More complex types
* Pattern matching

## Simple types

Types is one of the most important strength in F#. I've devided types into two sections, one around simple types and then one more advanced types later on.

Types are defined using the `type` keyword. That is used for all the different types you can define, records, discriminated unions, classes, abstract classes or interfaces (yes you can do classes and interfaces in F#)

Let's see some examples

In [ ]:
// Type alias 
type Age = int // This is just an alias over a native built-in type

// Record
type Person = {
    Name: string
    Age: int
}

// Discriminated union
type Employee =
    | Manager of Person
    | Employee of Person

## All you need to know - `let`

`let` is the keyword used to define and assign a variable in F#. You use it to define functions and values. One thing that is important in F# is that everything is an expression. So everything has an output.

In [ ]:
// Integer
let x = 10
// String
let y = "Hello"
// Record 
let tomas = { Name = "Tomas"; Age = 30 }
let tomas2 = {
    Name = "Tomas" // No need for ;
    Age = 30
}
// DUs
let employee = Employee tomas
let manager = Employee tomas2

// Functions
let add x y = x + y
add 5 6
// add 5.5 5.6 // Fails since add has been inferred to be of type int

11

In [ ]:
// More functions
let changeName name person = { person with Name = name }
changeName "Ole" tomas


Name,Age
Tomas,30


In [ ]:
tomas // is still unchanged

Name,Age
Tomas,30


In [ ]:
let changeEmployeeName name employee = 
    match employee with
    | Manager person -> Manager (changeName name person)
    | Employee person -> Employee (changeName name person)

changeEmployeeName "ole" manager

Item
"{ { Name = ""ole"" Age = 30 }: Name: ole, Age: 30 }"


## Currying and partial applications

Currying and partial applications are central concept in programming in general, and you can also use it in most languages today that treats functions as first class citizens (C# does as well).

Currying is to allow a function that takes multiple arguments to be written as a sequence of functions where each function takes one argument.

    let add1 (x,y) = x + y // function that takes to arguments
    let add2 x y = x + y // curried version

Partial applications is what you got when you call a curried function with only some of its values

    let add100 = add2 100 // This will call add2 with argumenet 100 returning a new function where x is set to be 100

In [ ]:
let add1 (x,y) = x + y
let add2 x y = x + y
let add100 = add2 100

(add1(5,4), add 5 4, add100 5)

Item1,Item2,Item3
9,9,105


## Piping (|>)

Piping is very commong in F#, and it is a lot like bash piping. The syntax in short is

    value |> function

this is exactly the same as calling

    function value